In [236]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from multiprocessing import Pool
import FinanceDataReader as fdr
import requests, lxml
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

import time, datetime

In [100]:
def getInvesting(ticker):
    target_address = 'https://www.google.com/search?q='+ticker+'+investing'
    html = requests.get(target_address)
    soup = BeautifulSoup(html.text, 'html.parser')
    posts = soup.select('.kCrYT a ')
    #for chooes first herf 
    href = str(posts[0].attrs['href'])
    #then href is 'https:/www.investing/equities/stock&~~~~~~ 
    #so 1.find  start-index : index('equities') , end-index : index('&')
    # and slicing href[start : end]
    start = href.find('equities')
    end = href.find('&')
    # print(href[start:end])
    return "https://www.investing.com/" +href[start:end]+'-commentary'

In [21]:
def get_comments(target_address):
    result = []
    headers = {'User-Agent':"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.61 Safari/537.36"}
    html = requests.get(target_address)
    soup = BeautifulSoup(html.text, 'html.parser')
    print(soup)
    posts = soup.find_all('div', {'class' : 'comment_content__AvzPV'})
    for i in posts:
        result.append(i.text.strip())
    return result

In [335]:
titles = []
dates = []

def get_title(corp):
    
    options = webdriver.ChromeOptions()
    options.add_experimental_option('excludeSwitches', ['enable-logging'])
    options.add_argument('user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.61 Safari/537.36')
    driver = webdriver.Chrome(executable_path='./chromedriver', options=options)
    
    driver.get(getInvesting(corp))
    driver.implicitly_wait(2)

    articles = driver.find_elements_by_class_name('comment_content__AvzPV')
    soup = BeautifulSoup(driver.page_source, 'lxml')
    
    for line in articles:
        titles.append(line.text)

    for line in soup.select('div.comment_chat-header__36SjM span span'):
        title = line.text
        dates.append(title)        

    page = 0
    cnt = 0
    
    while True:
        page += 2
        
        if page % 10 == 0:
            time.sleep(10)
        
        url = getInvesting(corp)+ '/' + str(page)
        
        driver.get(url)
        driver.implicitly_wait(10)

        articles = driver.find_elements_by_class_name('comment_content__AvzPV')
        soup = BeautifulSoup(driver.page_source, 'lxml')
        link = soup.find_all('link', rel='canonical')
        
        if link[0]['href'] == getInvesting(corp):
            print(cnt)
            
            break

        for line in articles:
            titles.append(line.text)
            
        for line in soup.select('div.comment_chat-header__36SjM span span'):
            title = line.text
            dates.append(title)
        
        cnt += 1
        
    driver.close()

    return titles, dates

In [336]:
result = get_title('TSLA')

df = pd.DataFrame(
    {'DATE' : result[1],
    'COMMENT' : result[0]})

df.to_csv('TESLA.csv', index=False, encoding='UTF-8')

/var/folders/xw/rbcyfp692g3c5w_0qcls7ncm0000gn/T/ipykernel_21490/3144339217.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='./chromedriver', options=options)
/var/folders/xw/rbcyfp692g3c5w_0qcls7ncm0000gn/T/ipykernel_21490/3144339217.py:14: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  articles = driver.find_elements_by_class_name('comment_content__AvzPV')
/var/folders/xw/rbcyfp692g3c5w_0qcls7ncm0000gn/T/ipykernel_21490/3144339217.py:38: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  articles = driver.find_elements_by_class_name('comment_content__AvzPV')


NoSuchWindowException: Message: no such window: window was already closed
  (Session info: chrome=102.0.5005.61)
Stacktrace:
0   chromedriver                        0x0000000102eb9170 chromedriver + 4477296
1   chromedriver                        0x0000000102e50c18 chromedriver + 4049944
2   chromedriver                        0x0000000102aa2d40 chromedriver + 191808
3   chromedriver                        0x0000000102a937d8 chromedriver + 128984
4   chromedriver                        0x0000000102a8de34 chromedriver + 106036
5   chromedriver                        0x0000000102afa078 chromedriver + 548984
6   chromedriver                        0x0000000102ac6f58 chromedriver + 339800
7   chromedriver                        0x0000000102e8ebd4 chromedriver + 4303828
8   chromedriver                        0x0000000102e93094 chromedriver + 4321428
9   chromedriver                        0x0000000102e97600 chromedriver + 4339200
10  chromedriver                        0x0000000102e93af4 c

In [225]:
dates = []

options = webdriver.ChromeOptions()
options.add_experimental_option('excludeSwitches', ['enable-logging'])
options.add_argument('user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.61 Safari/537.36')
driver = webdriver.Chrome(executable_path='./chromedriver', options=options)

driver.get(getInvesting('TSLA'))
driver.implicitly_wait(2)

soup = BeautifulSoup(driver.page_source, 'lxml')

for line in soup.select('div.comment_chat-header__36SjM span span'):
    title = line.text
    dates.append(title)

#comments > ul > li:nth-child(23) > div > div > div.comment_chat-header__36SjM > span > span
    
# dates = driver.find_element(by=By.CLASS_NAME, value='comment_user-info__AWjKG')
# print(dates)
# print(dates.text)
#comments > ul > li:nth-child(24) > div > div > div.comment_chat-header__36SjM > span > span
# for line in date:
#     dates.append(line.text)

/var/folders/xw/rbcyfp692g3c5w_0qcls7ncm0000gn/T/ipykernel_21490/469981134.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='./chromedriver', options=options)


In [226]:
dates

['30 minutes ago',
 '1 hour ago',
 '11 hours ago',
 '10 hours ago',
 '10 hours ago',
 '11 hours ago',
 '12 hours ago',
 '12 hours ago',
 '13 hours ago',
 '12 hours ago',
 '12 hours ago',
 '13 hours ago',
 '8 hours ago',
 '14 hours ago',
 '15 hours ago',
 '13 hours ago',
 '15 hours ago',
 '15 hours ago',
 '15 hours ago',
 '16 hours ago',
 '17 hours ago',
 '17 hours ago',
 '17 hours ago',
 '16 hours ago',
 '17 hours ago',
 '17 hours ago',
 '17 hours ago',
 '17 hours ago',
 '17 hours ago',
 '18 hours ago',
 '18 hours ago',
 '17 hours ago',
 '16 hours ago',
 '19 hours ago',
 '18 hours ago',
 '18 hours ago',
 '19 hours ago',
 '22 hours ago',
 '21 hours ago',
 '23 hours ago',
 '23 hours ago']